1. Setup


1.1 Install dependencies

In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

1.2 import dependencies

In [1]:
#import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

#import tensorflow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

1.3 Set GPU growth

In [2]:
#Avoid OOM errors by setting gpu consumption limit
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu,True)

1.4 Create folder structure

In [3]:
# setup paths
POS_PATH= os.path.join('data','positive')
NEG_PATH= os.path.join('data','negative')
ANC_PATH= os.path.join('data','anchor')

In [4]:
#make directories
# os.makedirs(POS_PATH)
# os.makedirs(NEG_PATH)
# os.makedirs(ANC_PATH)

2.Collecting data

collect face database

In [5]:
#http://vis-www.cs.umass.edu/lfw

In [6]:
#uncompress the face data
#!tar -xf lfw.tgz

In [10]:
# #move images into negative file data
# for directory in os.listdir('lfw'):
#     for file in os.listdir(os.path.join('lfw',directory)):
#         EX_PATH = os.path.join('lfw',directory,file)
#         NEW_PATH = os.path.join(NEG_PATH,file)
#         os.replace(EX_PATH,NEW_PATH)

2. Collect positive and anchor cases

In [7]:
#import uuid to generate unique image names
import uuid

In [22]:
#establish a connection to webcam
cap= cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    #cut down frame
    frame = frame[120:120+250,200:200+250,:]

    #collect anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname,frame)
        
    #collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname,frame)
        
    #show image on screen
    cv2.imshow('Image Collection',frame)

    #breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
#release the webcam
cap.release()
cv2.destroyAllWindows()

3.Load and preprocess

3.1 Get image directories

In [8]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(300)

3.2 preprocess

In [9]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

3.3 Create labelled dataset

In [10]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

3.4 Create train and test partitions

In [11]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [12]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)

In [13]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [14]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

4. Model engineering

4.1 Build embedding layer

In [15]:
def make_embedding():
    inp = Input(shape=(100,100,3),name='input image')

    #first block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    #second block
    c2 = Conv2D(128, (7,7),activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2),padding='same')(c2)

    #third block
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2),padding='same')(c3)

    #final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096,activation='sigmoid')(f1)


    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [16]:
embedding = make_embedding()

In [17]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input image (InputLayer)     [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 91, 91, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0 

4.2 Make a distance layer

In [18]:
#building custom layer
class L1Dist(Layer):
    def __init__(self,**kwargs):
        super().__init__()
    #where it happens
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)


4.3 Make siamese model

In [32]:
def make_siamese_model():
    #handle inputs
    input_image = Input(name='input image', shape=(100,100,3))
    validation_image = Input(name='validation_image',shape=(100,100,3))

    #combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image),embedding(validation_image))

    #classification layer
    classifier = Dense(1,activation='sigmoid')(distances)

    return Model(inputs=[input_image, validation_image], outputs=classifier,name='SiameseNetwork')


In [34]:
siamese_model = make_siamese_model()

In [35]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input image (InputLayer)        [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_image (InputLayer)   [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input image[0][0]                
                                                                 validation_image[0][0]           
__________________________________________________________________________________________________
distance (L1Dist)               (None, 4096)         0           embedding[2][0]     

5. Train the neural network

5.1 Setup loss function

In [36]:
#Network tries to minimise the loss function
#Get loss function from tensorflow
binary_cross_loss = tf.losses.BinaryCrossentropy()

5.1.2 Define the optimiser

In [37]:
#Define optimiser
opt = tf.keras.optimizers.Adam(1e-4)

5.2 Establish checkpoints

In [45]:
#setup checkpoint directory
checkpoint_dir = './training_checkpoints'

#create prefix for each checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir,'ckpt')

#create checkpoint
checkpoint = tf.train.Checkpoint(opt=opt,siamese_model=siamese_model)


5.3 Build a custom training step

In [46]:
@tf.function
def train_step(batch):

    with tf.GradientTape() as tape:
        #get anchor and validation image
        X = batch[:2]
        #get label
        y= batch[2]

        #forward pass
        yhat = siamese_model(X, training=True)
        #calculate loss
        loss = binary_cross_loss(y,yhat)

    
    
    #calculate gradients
    grad = tape.gradient(loss,siamese_model.trainable_variables)

    #calculate updated weights and apply
    opt.apply_gradients(zip(grad,siamese_model.trainable_variables))

    

    

5.4 Define a training loop

In [54]:
def train(data,EPOCHS):
    #loop through the epochs
    for epoch in range(1,EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch,EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))

        #loop through each batch
        for idx,batch in enumerate(data):
            #run train step
            train_step(batch)
            progbar.update(idx+1)

        #save checkpoints
        if epoch%10 ==0:
            checkpoint.save(file_prefix=checkpoint_prefix)

    


5.5 Train the model

In [55]:
EPOCHS = 50

In [56]:
train(train_data,EPOCHS)


 Epoch 1/50
27/27 [==============================] - 509s 19s/step

 Epoch 2/50
27/27 [==============================] - 508s 19s/step

 Epoch 3/50
27/27 [==============================] - 508s 19s/step

 Epoch 4/50
27/27 [==============================] - 508s 19s/step

 Epoch 5/50
27/27 [==============================] - 509s 19s/step

 Epoch 6/50
27/27 [==============================] - 509s 19s/step

 Epoch 7/50
27/27 [==============================] - 507s 19s/step

 Epoch 8/50
27/27 [==============================] - 510s 19s/step

 Epoch 9/50
27/27 [==============================] - 510s 19s/step

 Epoch 10/50
27/27 [==============================] - 508s 19s/step

 Epoch 11/50
27/27 [==============================] - 509s 19s/step

 Epoch 12/50
27/27 [==============================] - 508s 19s/step

 Epoch 13/50
27/27 [==============================] - 507s 19s/step

 Epoch 14/50
27/27 [==============================] - 507s 19s/step

 Epoch 15/50
27/27 [======================